# Finetuning

In [ ]:
"""
CUAD Long-Context Fine-tuning using Unsloth - Optimized for 65k tokens
Based on SQuAD-style formatting for generative QA tasks
"""


'\nCUAD Long-Context Fine-tuning using Unsloth - Optimized for 65k tokens\nBased on SQuAD-style formatting for generative QA tasks\n'

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Colab-specific installation - optimized versions
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install transformers==4.51.3
    !pip install --no-deps unsloth

In [ ]:
# # Upgrade datasets for better long-context handling
# !pip install --upgrade datasets fsspec

In [ ]:
# Import required libraries
from unsloth import FastLanguageModel, UnslothTrainer, UnslothTrainingArguments
from unsloth.chat_templates import train_on_responses_only
import torch
from datasets import load_dataset
from transformers import DataCollatorForLanguageModeling
from unsloth import is_bfloat16_supported
import time
from datetime import datetime
import gc
import json
import numpy as np

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
print("="*80)
print("CUAD LONG-CONTEXT FINE-TUNING - LLAMA 3.2 3B")
print("Optimized for up to 65k token contexts")
print("="*80)
print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Model configuration optimized for A100 40GB with long contexts
max_seq_length = 65536  # Full 65k context length
dtype = None  # Auto detection - will use bfloat16 on A100
load_in_4bit = True

print("\n1. Loading Llama 3.2 3B model with long-context support...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)
tokenizer.model_max_length = max_seq_length
tokenizer.init_kwargs["model_max_length"] = max_seq_length

print("2. Adding LoRA adapters - optimized for long-context QA...")
# LoRA configuration following Unsloth best practices
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Balanced rank - good for QA tasks # increase for increasing tuning of more parameters
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                   "gate_proj", "up_proj", "down_proj",],
                  #  "embed_tokens", "lm_head"],  # Include embedding layers for better adaptation
    lora_alpha=32,  # 2x the rank as per best practices
    lora_dropout=0,  # Optimized setting per Unsloth
    bias="none",  # Optimized setting per Unsloth
    use_gradient_checkpointing="unsloth",  # Unsloth's memory-efficient version
    random_state=3407,
    use_rslora=True,  # Rank stabilized LoRA for better training
    loftq_config=None,
)

# Explicitly set tokenizer parameters for long context
tokenizer.model_max_length = max_seq_length
tokenizer.pad_token = tokenizer.eos_token
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

print(f"Tokenizer max length set to: {tokenizer.model_max_length}")
print(f"Model max position embeddings: {model.config.max_position_embeddings}")

CUAD LONG-CONTEXT FINE-TUNING - LLAMA 3.2 3B
Optimized for up to 65k token contexts
Started at: 2025-06-07 12:05:39

1. Loading Llama 3.2 3B model with long-context support...
==((====))==  Unsloth 2025.6.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
2. Adding LoRA adapters - optimized for long-context QA...


Unsloth 2025.6.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


Tokenizer max length set to: 65536
Model max position embeddings: 131072


In [ ]:
def preprocess_cuad_function(examples):
    """
    Convert CUAD dataset to SQuAD-style format for generative QA.
    Handles empty answers properly for legal document analysis.
    """
    contexts = examples["context"]
    questions = examples["question"]
    answers = []

    # Process answers - handle the SQuAD-style answer format and empty answers
    for ans in examples["answers"]:
        if ans.get("text") and len(ans["text"]) > 0:
            # Take the first non-empty answer, strip whitespace
            answer_text = next((a.strip() for a in ans["text"] if a.strip()), "")
            # If still empty after stripping, use "Not found"
            answers.append(answer_text if answer_text else "Not found")
        else:
            answers.append("Not found")  # Consistent format for empty answers

    # Create input texts - improved prompt structure
    inputs = [
        f"You are a legal document analyzer. Extract exact phrases from legal documents to answer questions. Only provide the exact text/phrases from the document that answer the question. Do not add explanations or commentary. If the information is not found, respond with 'Not found'.\n\nDocument: {ctx}\n\nQuestion: {q}\n\nAnswer (extract exact phrase from document):"
        for ctx, q in zip(contexts, questions)
    ]

    return {
        "input_text": inputs,
        "target_text": answers,
    }

def formatting_prompts_func(example):
    """
    Format examples for training following Unsloth tutorial pattern.
    This creates the proper training format with end-of-text tokens.
    """
    # Handle both single examples and batched examples
    if isinstance(example["input_text"], str):
        # Single example
        input_text = example["input_text"]
        target_text = example["target_text"]
        # Format: Input + space + target + end-of-text token
        formatted_text = f"{input_text} {target_text}<|eot_id|>"
        return [formatted_text]
    else:
        # Batched examples - return list of formatted strings
        formatted_texts = []
        for inp, tgt in zip(example["input_text"], example["target_text"]):
            formatted_text = f"{inp} {tgt}<|eot_id|>"
            formatted_texts.append(formatted_text)
        return formatted_texts

print("\n3. Loading and preprocessing CUAD dataset...")
try:
    # Load the full CUAD dataset
    cuad_dataset = load_dataset("theatticusproject/cuad-qa", trust_remote_code=True)
    print(f"Loaded {len(cuad_dataset['train'])} training examples")

    # For initial testing, you might want to use a subset
    # Uncomment the next two lines for faster experimentation
    cuad_dataset["train"] = cuad_dataset["train"].shuffle(seed=42).select(range(10000))
    print(f"Using subset of {len(cuad_dataset['train'])} examples for faster training")

except Exception as e:
    print(f"Error loading CUAD dataset: {e}")
    raise


3. Loading and preprocessing CUAD dataset...
Loaded 22450 training examples
Using subset of 10000 examples for faster training


In [ ]:
print("4. Preprocessing dataset to SQuAD format...")
try:
    # Apply preprocessing to convert to input_text/target_text format
    cuad_processed = cuad_dataset.map(
        preprocess_cuad_function,
        batched=True,
        num_proc=4,  # Parallel processing
        remove_columns=cuad_dataset["train"].column_names,
        desc="Converting to SQuAD format"
    )

    print(f"Processed dataset size: {len(cuad_processed['train'])} examples")

    # Check for empty examples and filter them out
    def filter_empty_examples(example):
        return (len(example["input_text"].strip()) > 100 and  # Minimum context length
                len(example["target_text"].strip()) > 0)      # Must have some answer (including "Not found")

    cuad_processed["train"] = cuad_processed["train"].filter(filter_empty_examples)
    print(f"After filtering: {len(cuad_processed['train'])} examples")
    train_dataset = cuad_processed["train"]  # Store reference for later use

except Exception as e:
    print(f"Error preprocessing dataset: {e}")
    raise

# Show sample of the processed data
if len(cuad_processed["train"]) > 0:
    print("\nSample processed example:")
    print("-" * 80)
    sample = cuad_processed["train"][0]
    print("Input text (first 500 chars):")
    print(sample["input_text"][:500] + "...")
    print(f"\nTarget text: {sample['target_text']}")
    print("\nFormatted example (first 600 chars):")
    formatted_sample = formatting_prompts_func(sample)[0]
    print(formatted_sample[:600] + "...")
    print("-" * 80)
    print(formatted_sample[-400:])

4. Preprocessing dataset to SQuAD format...
Processed dataset size: 10000 examples
After filtering: 10000 examples

Sample processed example:
--------------------------------------------------------------------------------
Input text (first 500 chars):
You are a legal document analyzer. Extract exact phrases from legal documents to answer questions. Only provide the exact text/phrases from the document that answer the question. Do not add explanations or commentary. If the information is not found, respond with 'Not found'.

Document: Exhibit 10.1   JACKSONVILLE JAGUARS SPONSORSHIP AGREEMENT   This Sponsorship Agreement (this "Agreement") is entered into as of November 27, 2017 (the "Execution Date") by and between Jacksonville Jaguars, LLC, a...

Target text: Not found

Formatted example (first 600 chars):
You are a legal document analyzer. Extract exact phrases from legal documents to answer questions. Only provide the exact text/phrases from the document that answer the question. Do

In [ ]:
# Forcefully set the tokenizer's max length. This is still good practice.
tokenizer.model_max_length = max_seq_length
tokenizer.init_kwargs["model_max_length"] = max_seq_length

# This new, single function will handle EVERYTHING: formatting, tokenizing, and masking.
def prepare_and_tokenize_dataset(examples):
    """
    This function takes a batch of examples, formats them into the full prompt-answer
    string, tokenizes them to the full max_seq_length, and creates the masked labels.
    """
    # This is the boundary marker for masking
    answer_prefix = "Answer (extract exact phrase from document):"

    # Format the full text string (prompt + answer + EOS)
    # The `formatting_prompts_func` returns a list, so we access the first element.
    texts = formatting_prompts_func(examples) # This will return a list of formatted strings

    # Tokenize the full texts to the max sequence length
    model_inputs = tokenizer(
        texts,
        truncation=True,
        padding=False, # This is essential!
        max_length=max_seq_length,
    )

    # Create the labels and apply the mask.
    # We will mask everything up to and including the answer_prefix.
    labels = []
    for i in range(len(model_inputs["input_ids"])):
        # Find the start of the answer
        # We search for the tokenized representation of our answer prefix
        # Note: We search in the input_ids of this specific example.

        # We need to find the end of the prompt to mask it.
        # Let's find the start of the answer by finding the answer_prefix tokens
        # We need to tokenize the prefix itself to find its token IDs
        # Note: add_special_tokens=False is important here!
        prefix_tokens = tokenizer.encode(answer_prefix, add_special_tokens=False)

        # Find the sequence of prefix_tokens in the input_ids
        input_ids = model_inputs["input_ids"][i]
        prompt_end_idx = -1
        for k in range(len(input_ids) - len(prefix_tokens)):
            if input_ids[k:k+len(prefix_tokens)] == prefix_tokens:
                prompt_end_idx = k + len(prefix_tokens)
                break

        # Create a copy of the input_ids to use as labels
        label = list(input_ids)

        if prompt_end_idx != -1:
            # Mask everything up to the end of the prefix
            # The +1 is to account for the space token after the prefix
            label[:prompt_end_idx ] = [-100] * (prompt_end_idx )
        else:
            # If for some reason the prefix isn't found, mask the whole sequence
            # This prevents accidental training on prompts if something goes wrong.
            label[:] = [-100] * len(label)

        labels.append(label)

    # Add the labels to our model inputs
    model_inputs["labels"] = labels

    return model_inputs

# --- Apply this new function to the dataset ---
print("Preparing and tokenizing dataset manually to full context length...")
cuad_tokenized = cuad_processed.map(
    prepare_and_tokenize_dataset,
    batched=True,
    # This function is complex, so we run it on a single process.
    num_proc=4,
    remove_columns=cuad_processed["train"].column_names,
    desc="Tokenizing and Masking all examples"
)

# --- Verification Step ---
print("\nVerifying tokenized output...")
print("Input IDs shape of first example:", len(cuad_tokenized['train'][0]['input_ids']))
print("Labels shape of first example:", len(cuad_tokenized['train'][0]['labels']))

Preparing and tokenizing dataset manually to full context length...

Verifying tokenized output...
Input IDs shape of first example: 3654
Labels shape of first example: 3654


In [ ]:
print("\n5. Setting up training configuration...")

# Data collator for language modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Training arguments optimized for long-context and A100 40GB
trainer = UnslothTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=cuad_tokenized["train"],
    # dataset_text_field=None,  # This will be transformed by formatting_func
    # formatting_func=formatting_prompts_func,  # Key function for proper formatting
    max_seq_length=max_seq_length,
    data_collator=data_collator,
    args=UnslothTrainingArguments(
        # Batch size configuration for A100 40GB with 65k context
        per_device_train_batch_size=2,      # Start with 1 due to long sequences
        gradient_accumulation_steps=4,      # Effective batch size = 8

        # Learning rate configuration
        learning_rate=2e-4,                 # Standard Unsloth learning rate
        embedding_learning_rate=1e-5,       # Lower LR for embeddings (continual pretraining)

        # Training schedule
        num_train_epochs=1,                 # Start with 1 epoch for long contexts
        warmup_ratio=0.03,                  # Smaller warmup for long training

        # Optimization settings
        optim="adamw_8bit",                 # Memory-efficient optimizer
        weight_decay=0.01,
        lr_scheduler_type="cosine",         # Cosine scheduler as in reference
        max_grad_norm=1.0,

        # Precision settings for A100
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),

        # Logging and saving
        logging_steps=5,
        save_steps=100,
        save_total_limit=3,

        # Output configuration
        output_dir="./cuad_long_context_outputs",
        report_to="none",

        # Memory optimization
        dataloader_num_workers=2,
        remove_unused_columns=True,        # Keep for custom formatting

        # Reproducibility
        seed=3407,

    ),
)

# Mask everything up through the answer prompt, un-mask only the answer tokens
# trainer = train_on_responses_only(
#     trainer,
#     instruction_part="Answer (extract exact phrase from document):",
#     response_part   ="Answer (extract exact phrase from document):"
# )

# Memory check before training
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"\nGPU: {gpu_stats.name}")
print(f"Max memory: {max_memory} GB")
print(f"Memory reserved before training: {start_gpu_memory} GB")
print(f"Available memory: {max_memory - start_gpu_memory} GB")


5. Setting up training configuration...

GPU: NVIDIA A100-SXM4-40GB
Max memory: 39.557 GB
Memory reserved before training: 3.441 GB
Available memory: 36.116 GB


In [ ]:
# grab a single batch
batch = next(iter(trainer.get_train_dataloader()))
print("Input IDs shape:", batch["input_ids"].shape)
# Expect shape: [batch_size, max-seq-length in that batch]

# Also inspect one example
feature = trainer.train_dataset[0]
length_after_tokenization = len(feature["input_ids"])
print("Tokenized length of sample 0:", length_after_tokenization)

Input IDs shape: torch.Size([2, 23914])
Tokenized length of sample 0: 3654


In [ ]:
# Pick a sample index to inspect (0 for the first example)
idx = 0

# Grab the labels and input_ids for that example
feature = trainer.train_dataset[idx]
labels   = feature["labels"]      # –100 for masked tokens, real IDs for un-masked
input_ids= feature["input_ids"]

# Which token IDs survive the masking?
unmasked_ids = [l for l in labels if l != -100]

# Decode them back to text
decoded_answer = tokenizer.decode(unmasked_ids)

print("=== UNMASKED TOKENS (answer) ===")
print(decoded_answer)

# (Optional) see the full prompt for context
full_prompt = tokenizer.decode(input_ids)
print("\n=== FULL PROMPT ===")
print(full_prompt)


=== UNMASKED TOKENS (answer) ===
 Not found<|eot_id|>

=== FULL PROMPT ===
<|begin_of_text|>You are a legal document analyzer. Extract exact phrases from legal documents to answer questions. Only provide the exact text/phrases from the document that answer the question. Do not add explanations or commentary. If the information is not found, respond with 'Not found'.

Document: Exhibit 10.1   JACKSONVILLE JAGUARS SPONSORSHIP AGREEMENT   This Sponsorship Agreement (this "Agreement") is entered into as of November 27, 2017 (the "Execution Date") by and between Jacksonville Jaguars, LLC, a Delaware limited liability company ("Club"), and The ARC Group, Inc., a Florida corporation (owner and operator of Dick's Wings and Grill) ("Sponsor"). This Agreement consists of this Sponsorship Agreement and Exhibits A and B hereto, each of which is incorporated into and forms a part of this Agreement by this reference.   RECITALS   A. Club owns and operates the National Football League ("NFL") team kn

In [ ]:
print("\nVerifying the masking by decoding the first training example's labels...")

# 1. Grab the first processed example from the trainer's dataset
first_example_processed = trainer.train_dataset[1]

# 2. Get the list of labels. It will be a mix of token IDs and -100s.
labels = first_example_processed["labels"]

# 3. Filter out all the masked tokens (the -100s).
unmasked_label_ids = [label for label in labels if label != -100]

# 4. Decode the remaining token IDs back into a readable string.
decoded_answer = tokenizer.decode(unmasked_label_ids)

print("\n------------------------------------------------------------")
print("Decoded Unmasked Labels (This should ONLY be the answer):")
print(f"'{decoded_answer}'")
print("------------------------------------------------------------\n")

# You can also check the number of unmasked tokens
print(f"Total tokens in sequence: {len(labels)}")
print(f"Number of unmasked (answer) tokens: {len(unmasked_label_ids)}")

# ===================================================================================
# END OF VERIFICATION CODE
# ===================================================================================



Verifying the masking by decoding the first training example's labels...

------------------------------------------------------------
Decoded Unmasked Labels (This should ONLY be the answer):
' Boyd<|eot_id|>'
------------------------------------------------------------

Total tokens in sequence: 2038
Number of unmasked (answer) tokens: 2


In [ ]:
# Test the formatting function with a sample
print("\n6. Testing formatting function...")
test_sample = cuad_processed["train"][0]
formatted_test = formatting_prompts_func(test_sample)
print("Formatted training example (first 800 characters):")
print(formatted_test[:800] + "..." if len(formatted_test) > 800 else formatted_test)


6. Testing formatting function...
Formatted training example (first 800 characters):
['You are a legal document analyzer. Extract exact phrases from legal documents to answer questions. Only provide the exact text/phrases from the document that answer the question. Do not add explanations or commentary. If the information is not found, respond with \'Not found\'.\n\nDocument: Exhibit 10.1   JACKSONVILLE JAGUARS SPONSORSHIP AGREEMENT   This Sponsorship Agreement (this "Agreement") is entered into as of November 27, 2017 (the "Execution Date") by and between Jacksonville Jaguars, LLC, a Delaware limited liability company ("Club"), and The ARC Group, Inc., a Florida corporation (owner and operator of Dick\'s Wings and Grill) ("Sponsor"). This Agreement consists of this Sponsorship Agreement and Exhibits A and B hereto, each of which is incorporated into and forms a part of this Agreement by this reference.   RECITALS   A. Club owns and operates the National Football League ("NFL") team k

In [ ]:
print("\n7. Starting training...")
print("=" * 60)
start_time = time.time()

try:
    # Clear cache before training
    torch.cuda.empty_cache()
    gc.collect()

    # Start training
    trainer_stats = trainer.train()

    training_time = time.time() - start_time
    final_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)

    print("=" * 60)
    print("TRAINING COMPLETED SUCCESSFULLY!")
    print(f"Training time: {training_time/60:.2f} minutes")
    print(f"Peak GPU memory: {final_gpu_memory} GB")
    print(f"Memory increase: {final_gpu_memory - start_gpu_memory} GB")

    if trainer.state.log_history:
        final_loss = trainer.state.log_history[-1].get('train_loss', 'N/A')
        print(f"Final training loss: {final_loss}")

    training_successful = True

except Exception as e:
    print(f"Training failed: {e}")
    print("This might be due to memory constraints with very long sequences.")
    print("Consider reducing per_device_train_batch_size to 1 or max_seq_length.")
    training_successful = False
    raise


7. Starting training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,000 | Num Epochs = 1 | Total steps = 1,250
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
5,4.045800
10,4.977900
15,3.401000
20,3.870600
25,1.900300
30,2.285800
35,2.414500
40,1.878500
45,1.466200
50,1.491700


TRAINING COMPLETED SUCCESSFULLY!
Training time: 778.85 minutes
Peak GPU memory: 19.061 GB
Memory increase: 15.620000000000001 GB
Final training loss: 0.47625361874103544


In [ ]:
# Post-training code
if training_successful:
    end_time = time.time()
    training_time = end_time - start_time

    # Final memory stats - FOLLOWING UNSLOTH TUTORIAL FORMAT
    used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
    used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
    used_percentage = round(used_memory / max_memory * 100, 3)
    lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)

    print("=" * 40)
    print("TRAINING COMPLETED SUCCESSFULLY!")
    print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
    print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
    print(f"Peak reserved memory = {used_memory} GB.")
    print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
    print(f"Peak reserved memory % of max memory = {used_percentage} %.")
    print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

    print("\n8. Saving fine-tuned model...")

    # Save LoRA adapters - FOLLOWING UNSLOTH TUTORIAL
    model_save_path = "./cuad_finetuned_llama3_2_3b"
    model.save_pretrained(model_save_path)
    tokenizer.save_pretrained(model_save_path)

    # Save training info
    training_info = {
        "model_name": "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
        "dataset": "cuad",
        "training_examples": len(train_dataset),
        "max_seq_length": max_seq_length,
        "training_steps": trainer_stats.metrics.get('train_steps', 'unknown'),
        "training_time_minutes": training_time/60,
        "final_loss": trainer_stats.metrics.get('train_loss', 'unknown'),
        "saved_at": datetime.now().isoformat(),
        "lora_config": {
            "r": 16,  # Updated to match the actual config from first part
            "lora_alpha": 32,  # Updated to match the actual config from first part
            "lora_dropout": 0
        }
    }

    with open(f"{model_save_path}/training_info.json", "w") as f:
        json.dump(training_info, f, indent=2)

    print(f"Model saved to: {model_save_path}")
    print(f"Training info saved to: {model_save_path}/training_info.json")

    tokenizer.pad_token_id = tokenizer.eos_token_id  # or tokenizer.unk_token_id

    print("\n9. Quick inference test...")
    FastLanguageModel.for_inference(model)

    test_context = """
    This Software License Agreement ("Agreement") is entered into on January 1, 2024,
    between Company A and Company B. The term of this Agreement shall be for a period
    of three (3) years from the Effective Date, unless terminated earlier in accordance
    with the terms herein.
    """

    test_question = "What is the duration of this agreement?"

    # Use the same prompt format as in preprocessing
    test_prompt = f"You are a legal document analyzer. Extract exact phrases from legal documents to answer questions. Only provide the exact text/phrases from the document that answer the question. Do not add explanations or commentary. If the information is not found, respond with 'Not found'.\n\nDocument: {test_context}\n\nQuestion: {test_question}\n\nAnswer (extract exact phrase from document):"

    max_new_tokens = 64
    tokenized = tokenizer(
        test_prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_seq_length - max_new_tokens
    ).to("cuda")

    print(f"Test Question: {test_question}")
    print("Model Response:")

    with torch.no_grad():
        outputs = model.generate(
            **tokenized,
            max_new_tokens=max_new_tokens,
            use_cache=True,
            temperature=1.5,
            min_p=0.1,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    # Decode only the generated part (skip the input prompt)
    generated_text = tokenizer.decode(outputs[0][tokenized['input_ids'].shape[1]:], skip_special_tokens=True)
    print(generated_text)

    print("\n" + "="*80)
    print("FINE-TUNING COMPLETED SUCCESSFULLY!")
    print("="*80)
    print(f"Model saved at: {model_save_path}")
    print("You can now use the evaluation script to test performance.")
    print("="*80)

else:
    print("Training failed. Please check the error messages above.")

print(f"\nScript completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

TRAINING COMPLETED SUCCESSFULLY!
46728.6389 seconds used for training.
778.81 minutes used for training.
Peak reserved memory = 19.061 GB.
Peak reserved memory for training = 15.62 GB.
Peak reserved memory % of max memory = 48.186 %.
Peak reserved memory for training % of max memory = 39.487 %.

8. Saving fine-tuned model...
Model saved to: ./cuad_finetuned_llama3_2_3b
Training info saved to: ./cuad_finetuned_llama3_2_3b/training_info.json

9. Quick inference test...
Test Question: What is the duration of this agreement?
Model Response:
 The term of this Agreement shall be for a period 
    of three (3) years from the Effective Date, unless terminated earlier in accordance 
    with the terms herein.

FINE-TUNING COMPLETED SUCCESSFULLY!
Model saved at: ./cuad_finetuned_llama3_2_3b
You can now use the evaluation script to test performance.

Script completed at: 2025-06-08 01:05:56


In [ ]:
!zip -r llama_finetuned.zip /content/cuad_finetuned_llama3_2_3b

  adding: content/cuad_finetuned_llama3_2_3b/ (stored 0%)
  adding: content/cuad_finetuned_llama3_2_3b/tokenizer.json (deflated 85%)
  adding: content/cuad_finetuned_llama3_2_3b/adapter_config.json (deflated 55%)
  adding: content/cuad_finetuned_llama3_2_3b/README.md (deflated 66%)
  adding: content/cuad_finetuned_llama3_2_3b/training_info.json (deflated 34%)
  adding: content/cuad_finetuned_llama3_2_3b/special_tokens_map.json (deflated 63%)
  adding: content/cuad_finetuned_llama3_2_3b/tokenizer_config.json (deflated 94%)
  adding: content/cuad_finetuned_llama3_2_3b/adapter_model.safetensors (deflated 7%)


In [ ]:
from google.colab import files
files.download('llama_finetuned.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Evaluation

In [ ]:
"""
CUAD-QA Evaluation Script for Llama 3.2 3B - ALIGNED WITH FINE-TUNING
Supports both original and fine-tuned models
Uses SQuAD-style evaluation metrics
Matches the exact prompt format and parameters used during fine-tuning
"""

import os
import re
import string
import json
import argparse
import torch
import time
from collections import Counter
from datasets import load_dataset
from tqdm import tqdm
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# ================== EVALUATION METRICS ==================

def normalize_answer(s):
    """Normalize answer for comparison (SQuAD-style)"""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
    """Get normalized tokens from string"""
    if not s:
        return []
    return normalize_answer(s).split()

def exact_match_score(prediction, ground_truth):
    """Calculate exact match score"""
    return int(normalize_answer(prediction) == normalize_answer(ground_truth))

def f1_score(prediction, ground_truth):
    """Calculate F1 score between prediction and ground truth"""
    pred_tokens = get_tokens(prediction)
    truth_tokens = get_tokens(ground_truth)

    if len(pred_tokens) == 0 and len(truth_tokens) == 0:
        return 1.0
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return 0.0

    common_tokens = Counter(pred_tokens) & Counter(truth_tokens)
    num_same = sum(common_tokens.values())

    if num_same == 0:
        return 0.0

    precision = 1.0 * num_same / len(pred_tokens)
    recall = 1.0 * num_same / len(truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)

    return f1

def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    """Calculate max metric over all ground truths"""
    if not ground_truths:
        return 0.0
    scores = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores.append(score)
    return max(scores)

# ================== MODEL WRAPPER - ALIGNED WITH FINE-TUNING ==================

class CUADEvaluator:
    def __init__(self, model_path, max_seq_length=65536):
        self.model_path = model_path
        self.max_seq_length = max_seq_length
        self.model = None
        self.tokenizer = None
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def load_model(self):
        """Load model and tokenizer - ALIGNED WITH FINE-TUNING SCRIPT"""
        print(f"Loading model from: {self.model_path}")
        print(f"Max sequence length: {self.max_seq_length}")

        try:
            # Use same configuration as fine-tuning script
            self.model, self.tokenizer = FastLanguageModel.from_pretrained(
                model_name=self.model_path,
                max_seq_length=self.max_seq_length,
                dtype=None,  # Auto detection like in fine-tuning
                load_in_4bit=True,
                device_map="auto",
            )

            # Set pad_token_id like in fine-tuning script
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

            # Enable inference mode - FOLLOWING UNSLOTH TUTORIAL
            FastLanguageModel.for_inference(self.model)
            self.model.eval()

            print("✅ Model loaded successfully!")
            return True

        except Exception as e:
            print(f"❌ Error loading model: {e}")
            return False

    def clean_response(self, response):
        """Clean and extract answer from model response - IMPROVED"""
        if not response:
            return "Not found"

        # Remove common prefixes that models add
        prefixes_to_remove = [
            "Based on the provided contract",
            "Upon reviewing the contract",
            "The part of this contract",
            "The parts of this contract",
            "After analyzing the contract",
            "Upon analyzing the contract",
            "In this contract",
            "The contract",
            "Looking at",
            "According to",
            "From the contract",
            "The relevant",
            "Here is",
            "Here are",
            "The answer is",
            "Answer:",
            "Response:",
            "The exact phrase",
            "Extract exact phrase",
        ]

        cleaned = response.strip()

        # Remove common prefixes (case insensitive)
        for prefix in prefixes_to_remove:
            if cleaned.lower().startswith(prefix.lower()):
                cleaned = cleaned[len(prefix):].strip()
                # Remove common continuation patterns
                if cleaned.startswith(":"):
                    cleaned = cleaned[1:].strip()
                if cleaned.startswith(","):
                    cleaned = cleaned[1:].strip()
                break

        # Split by common separators and take the first substantive part
        separators = ['\n', '. ', ':', ' that should be reviewed', ' are:', ' is:']
        for sep in separators:
            if sep in cleaned:
                parts = cleaned.split(sep)
                if len(parts) > 1:
                    # Take the first non-empty meaningful part
                    for part in parts:
                        part = part.strip()
                        if part and len(part) > 3 and not part.lower().startswith(('the following', 'as follows')):
                            cleaned = part
                            break
                break

        # Final cleanup
        cleaned = cleaned.strip()

        # Remove quotes if they wrap the entire answer
        if cleaned.startswith('"') and cleaned.endswith('"'):
            cleaned = cleaned[1:-1].strip()
        if cleaned.startswith("'") and cleaned.endswith("'"):
            cleaned = cleaned[1:-1].strip()

        # If still too verbose or empty, return "Not found"
        if not cleaned or len(cleaned) > 200 or cleaned.lower().startswith(('there is no', 'no specific', 'i did not find', 'not found')):
            return "Not found"

        return cleaned

    def generate_answer(self, question, context, max_new_tokens=64):
        """Generate answer - EXACTLY MATCHING FINE-TUNING PROMPT FORMAT"""

        # Context truncation - same logic as fine-tuning
        max_context_length = self.max_seq_length - 1000
        if len(context) > max_context_length:
            # Keep beginning and end of context
            half_length = max_context_length // 2
            context = context[:half_length] + "\n[...TRUNCATED...]\n" + context[-half_length:]

        # EXACT SAME PROMPT FORMAT AS FINE-TUNING SCRIPT
        prompt = f"You are a legal document analyzer. Extract exact phrases from legal documents to answer questions. Only provide the exact text/phrases from the document that answer the question. Do not add explanations or commentary. If the information is not found, respond with 'Not found'.\n\nDocument: {context}\n\nQuestion: {question}\n\nAnswer (extract exact phrase from document):"

        try:
            # Tokenize with same approach as fine-tuning script
            tokenized = self.tokenizer(
                prompt,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=self.max_seq_length - max_new_tokens
            ).to(self.device)

            with torch.no_grad():
                outputs = self.model.generate(
                    **tokenized,
                    max_new_tokens=max_new_tokens,
                    use_cache=True,
                    temperature=1.5,  # Same as fine-tuning script inference test
                    min_p=0.1,       # Same as fine-tuning script inference test
                    do_sample=True,  # Same as fine-tuning script
                    pad_token_id=self.tokenizer.pad_token_id,
                    eos_token_id=self.tokenizer.eos_token_id,
                )

            # Decode only the generated part (skip the input prompt)
            generated_text = self.tokenizer.decode(
                outputs[0][tokenized['input_ids'].shape[1]:],
                skip_special_tokens=True
            ).strip()

            # Apply post-processing
            cleaned_response = self.clean_response(generated_text)

            return cleaned_response

        except Exception as e:
            print(f"Error during generation: {e}")
            return "Error generating response"

# ================== EVALUATION FUNCTIONS ==================

def evaluate_dataset(evaluator, dataset, max_samples=None, save_predictions=True, output_dir="./"):
    """Evaluate model on dataset"""

    if max_samples is not None:
        dataset = dataset.select(range(min(max_samples, len(dataset))))

    print(f"Evaluating on {len(dataset)} samples...")

    predictions = {}
    detailed_results = []
    total_em = 0
    total_f1 = 0

    start_time = time.time()

    for i, example in enumerate(tqdm(dataset, desc="Evaluating")):
        try:
            question_id = example.get('id', f"question_{i}")
            question = example['question']
            context = example['context']

            # Handle answers in CUAD format - same as fine-tuning preprocessing
            if example['answers']['text'] and len(example['answers']['text']) > 0:
                # Take the first non-empty answer, strip whitespace
                ground_truths = [next((a.strip() for a in example['answers']['text'] if a.strip()), "Not found")]
                # If still empty after stripping, use "Not found"
                if not ground_truths[0]:
                    ground_truths = ["Not found"]
            else:
                ground_truths = ["Not found"]  # Consistent with fine-tuning preprocessing

            # Generate prediction
            prediction = evaluator.generate_answer(question, context)
            predictions[question_id] = prediction

            # Calculate metrics
            em = metric_max_over_ground_truths(exact_match_score, prediction, ground_truths)
            f1 = metric_max_over_ground_truths(f1_score, prediction, ground_truths)

            total_em += em
            total_f1 += f1

            # Store detailed result
            detailed_results.append({
                'id': question_id,
                'question': question[:200] + "..." if len(question) > 200 else question,
                'context_length': len(context),
                'ground_truths': ground_truths,
                'prediction': prediction,
                'exact_match': em,
                'f1_score': f1
            })

            if (i + 1) % 10 == 0:
                current_em = (total_em / (i + 1)) * 100
                current_f1 = (total_f1 / (i + 1)) * 100
                print(f"Progress: {i+1}/{len(dataset)} | EM: {current_em:.2f}% | F1: {current_f1:.2f}%")

        except Exception as e:
            print(f"Error processing example {i}: {e}")
            continue

    end_time = time.time()
    evaluation_time = end_time - start_time

    # Calculate final metrics
    num_evaluated = len(detailed_results)
    if num_evaluated == 0:
        print("❌ No examples were successfully evaluated!")
        return None

    final_em = (total_em / num_evaluated) * 100
    final_f1 = (total_f1 / num_evaluated) * 100

    # Prepare results
    results = {
        'model_path': evaluator.model_path,
        'dataset_size': num_evaluated,
        'exact_match': final_em,
        'f1_score': final_f1,
        'evaluation_time_seconds': evaluation_time,
        'samples_per_second': num_evaluated / evaluation_time,
        'timestamp': datetime.now().isoformat(),
        'detailed_results': detailed_results,
    }

    # Print summary
    print("\n" + "="*80)
    print("EVALUATION RESULTS")
    print("="*80)
    print(f"Model: {evaluator.model_path}")
    print(f"Samples Evaluated: {num_evaluated}")
    print(f"Exact Match: {final_em:.2f}%")
    print(f"F1 Score: {final_f1:.2f}%")
    print(f"Evaluation Time: {evaluation_time/60:.2f} minutes")
    print(f"Speed: {num_evaluated/evaluation_time:.2f} samples/second")
    print("="*80)

    # Save results
    if save_predictions:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        model_name = os.path.basename(evaluator.model_path).replace("/", "_")

        # Save predictions
        pred_file = os.path.join(output_dir, f"predictions_{model_name}_{timestamp}.json")
        with open(pred_file, 'w') as f:
            json.dump(predictions, f, indent=2)

        # Save detailed results
        results_file = os.path.join(output_dir, f"results_{model_name}_{timestamp}.json")
        with open(results_file, 'w') as f:
            json.dump(results, f, indent=2)

        print(f"📁 Predictions saved to: {pred_file}")
        print(f"📁 Detailed results saved to: {results_file}")

    return results

# ================== MAIN FUNCTION ==================

def main():
    parser = argparse.ArgumentParser(description="Evaluate Llama 3.2 3B on CUAD dataset")
    parser.add_argument("--model_path",
                       default="unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
                       help="HuggingFace model repo or local path to fine-tuned model")
    parser.add_argument("--finetuned_model_path",
                       default="./cuad_finetuned_llama3_2_3b",
                       help="Path to fine-tuned model for comparison")
    parser.add_argument("--split",
                       default="test",
                       choices=["train", "test"],
                       help="Dataset split to evaluate on")
    parser.add_argument("--max_samples",
                       type=int,
                       default=4182,
                       help="Maximum number of samples to evaluate")
    parser.add_argument("--evaluate_original",
                       action="store_true",
                       default=True,
                       help="Evaluate original model")
    parser.add_argument("--evaluate_finetuned",
                       action="store_true",
                       default=True,
                       help="Evaluate fine-tuned model")
    parser.add_argument("--output_dir",
                       default="./evaluation_results",
                       help="Directory to save results")
    parser.add_argument("--max_seq_length",
                       type=int,
                       default=65536,
                       help="Maximum sequence length - should match fine-tuning")

    args, _ = parser.parse_known_args()

    # Create output directory
    os.makedirs(args.output_dir, exist_ok=True)

    print("🚀 CUAD EVALUATION SCRIPT - ALIGNED WITH FINE-TUNING")
    print("="*60)
    print(f"Split: {args.split}")
    print(f"Max samples: {args.max_samples}")
    print(f"Max sequence length: {args.max_seq_length}")
    print(f"Output directory: {args.output_dir}")

    # Load dataset
    print(f"\n📂 Loading CUAD {args.split} dataset...")
    try:
        dataset = load_dataset("theatticusproject/cuad-qa", split=args.split, trust_remote_code=True)
        print(f"✅ Loaded {len(dataset)} examples")
    except Exception as e:
        print(f"❌ Error loading dataset: {e}")
        return

    results_summary = []

    # Evaluate original model
    if args.evaluate_original:
        print(f"\n🔄 EVALUATING ORIGINAL MODEL")
        print("="*50)

        evaluator_original = CUADEvaluator(
            model_path=args.model_path,
            max_seq_length=args.max_seq_length,
        )

        if evaluator_original.load_model():
            original_results = evaluate_dataset(
                evaluator_original,
                dataset,
                max_samples=args.max_samples,
                output_dir=args.output_dir
            )
            if original_results:
                results_summary.append(('Original', original_results))

        # Clear memory
        del evaluator_original
        torch.cuda.empty_cache()

    # Evaluate fine-tuned model
    if args.evaluate_finetuned:
        print(f"\n🔄 EVALUATING FINE-TUNED MODEL")
        print("="*50)

        if not os.path.exists(args.finetuned_model_path):
            print(f"⚠️  Fine-tuned model not found at: {args.finetuned_model_path}")
            print("Skipping fine-tuned evaluation...")
        else:
            evaluator_finetuned = CUADEvaluator(
                model_path=args.finetuned_model_path,
                max_seq_length=args.max_seq_length,
            )

            if evaluator_finetuned.load_model():
                finetuned_results = evaluate_dataset(
                    evaluator_finetuned,
                    dataset,
                    max_samples=args.max_samples,
                    output_dir=args.output_dir
                )
                if finetuned_results:
                    results_summary.append(('Fine-tuned', finetuned_results))

            # Clear memory
            del evaluator_finetuned
            torch.cuda.empty_cache()

    # Compare results
    if len(results_summary) >= 2:
        print(f"\n📊 COMPARISON SUMMARY")
        print("="*80)
        print(f"{'Model':<15} {'Exact Match':<15} {'F1 Score':<15} {'Samples':<10} {'Time (min)':<12}")
        print("-" * 80)

        for model_name, results in results_summary:
            print(f"{model_name:<15} {results['exact_match']:<15.2f} "
                  f"{results['f1_score']:<15.2f} {results['dataset_size']:<10} "
                  f"{results['evaluation_time_seconds']/60:<12.2f}")

        # Calculate improvement
        if len(results_summary) == 2:
            original_em = results_summary[0][1]['exact_match']
            original_f1 = results_summary[0][1]['f1_score']
            finetuned_em = results_summary[1][1]['exact_match']
            finetuned_f1 = results_summary[1][1]['f1_score']

            em_improvement = finetuned_em - original_em
            f1_improvement = finetuned_f1 - original_f1

            print(f"\n🎯 IMPROVEMENT:")
            print(f"Exact Match: {em_improvement:+.2f} percentage points")
            print(f"F1 Score: {f1_improvement:+.2f} percentage points")

        print("="*80)

    print(f"\n✅ Evaluation completed! Results saved in: {args.output_dir}")

if __name__ == "__main__":
    main()

🚀 CUAD EVALUATION SCRIPT - ALIGNED WITH FINE-TUNING
Split: test
Max samples: 4182
Max sequence length: 65536
Output directory: ./evaluation_results

📂 Loading CUAD test dataset...
✅ Loaded 4182 examples

🔄 EVALUATING ORIGINAL MODEL
Loading model from: unsloth/Llama-3.2-3B-Instruct-bnb-4bit
Max sequence length: 65536
==((====))==  Unsloth 2025.6.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
✅ Model loaded successfully!
Evaluating on 4182 samples...


Evaluating:   0%|          | 10/4182 [00:19<1:40:54,  1.45s/it]

Progress: 10/4182 | EM: 40.00% | F1: 60.49%


Evaluating:   0%|          | 20/4182 [00:36<2:00:02,  1.73s/it]

Progress: 20/4182 | EM: 40.00% | F1: 54.25%


...

Progress: 4170/4182 | EM: 74.08% | F1: 75.97%


Evaluating: 100%|█████████▉| 4180/4182 [1:00:08<00:02,  1.49s/it]

Progress: 4180/4182 | EM: 74.04% | F1: 75.94%


Evaluating: 100%|██████████| 4182/4182 [1:00:09<00:00,  1.16it/s]


EVALUATION RESULTS
Model: ./cuad_finetuned_llama3_2_3b
Samples Evaluated: 4182
Exact Match: 74.06%
F1 Score: 75.95%
Evaluation Time: 60.15 minutes
Speed: 1.16 samples/second
📁 Predictions saved to: ./evaluation_results/predictions_cuad_finetuned_llama3_2_3b_20250608_053951.json
📁 Detailed results saved to: ./evaluation_results/results_cuad_finetuned_llama3_2_3b_20250608_053951.json

📊 COMPARISON SUMMARY
Model           Exact Match     F1 Score        Samples    Time (min)  
--------------------------------------------------------------------------------
Original        27.14           32.24           4182       167.00      
Fine-tuned      74.06           75.95           4182       60.15       

🎯 IMPROVEMENT:
Exact Match: +46.92 percentage points
F1 Score: +43.71 percentage points

✅ Evaluation completed! Results saved in: ./evaluation_results


In [ ]:
!zip -r evaluation_results.zip /content/evaluation_results

updating: content/evaluation_results/ (stored 0%)
  adding: content/evaluation_results/results_cuad_finetuned_llama3_2_3b_20250608_053951.json (deflated 93%)
  adding: content/evaluation_results/predictions_Llama-3.2-3B-Instruct-bnb-4bit_20250608_043934.json (deflated 85%)
  adding: content/evaluation_results/results_Llama-3.2-3B-Instruct-bnb-4bit_20250608_043934.json (deflated 92%)
  adding: content/evaluation_results/predictions_cuad_finetuned_llama3_2_3b_20250608_053951.json (deflated 91%)


In [ ]:
from google.colab import files
files.download('evaluation_results.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>